In [1]:
# Loading the Pallas module, Numpy and Matplotlib
import pallas_trace as pallas
import numpy as np
import matplotlib.pyplot as plt


trace_name = "bt.C.64_trace"

In [2]:
%%time
# Downloading a trace from a trusted source and decompressing it
from urllib.request import urlretrieve
import tarfile
urlretrieve(f"https://stark2.int-evry.fr/traces/pallas_traces/ABI_{pallas.get_ABI()}/{trace_name}.tgz", f"{trace_name}.tgz")
with tarfile.open(f"{trace_name}.tgz") as tarfile:
    tarfile.extractall(".")


<timed exec>:6: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.


CPU times: user 288 ms, sys: 208 ms, total: 496 ms
Wall time: 761 ms


In [2]:
%%time
if not trace_name.endswith(".pallas"):
    trace_name += "/eztrace_log.pallas"

trace = pallas.open_trace(trace_name)
# Checking out general information about the trace
print(f"Trace is located in {trace.dir_name}/{trace.trace_name}")
# This loads the archives
print(f"Trace contains {len(trace.archives)} archives")
# This loads the threads
print(f"Trace contains {sum([len(a.threads) for a in trace.archives])} threads")


Trace is located in /home/khatharsis/Documents/Benchmarks/software/src/pallas/docs/bt.C.64_trace/eztrace_log.pallas
Trace contains 64 archives
Trace contains 64 threads
CPU times: user 25.3 ms, sys: 30.4 ms, total: 55.7 ms
Wall time: 54.9 ms


In [4]:
%%time
# Manually
def print_pallas_object(obj: pallas.Sequence| pallas.Loop | pallas.EventSummary, index: int):
    return
    match type(obj):
        case pallas.Sequence:
            print(f"{obj.timestamps[index]/1e9}\t{obj.id}")
#        case pallas.Loop:
#            print(f"\tLoop {obj.id}")
        case pallas.EventSummary:
            print(f"{obj.timestamps[index]/1e9}\t{obj.id}")

def print_sequence(s: pallas.Sequence, counter: dict):
    for temp in s.content:
        if temp.id not in counter:
            counter[temp.id] = 0
        print_pallas_object(temp, counter[temp.id])
        counter[temp.id] +=1
        match type(temp):
            case pallas.Sequence:
                print_sequence(temp, counter)
            case pallas.Loop:
                for loop in range(temp.nb_iterations):
                    if temp.sequence.id not in counter:
                        counter[temp.sequence.id] = 0
                    print_pallas_object(temp.sequence, counter[temp.sequence.id])
                    print_sequence(temp.sequence, counter)
                    counter[temp.sequence.id] += 1

def print_thread(thread: pallas.Thread):
    counter = {}
    print_sequence(thread.sequences[0], counter)
                

print_thread(trace.archives[0].threads[0])
print("Done")

Done
CPU times: user 5.76 s, sys: 3.11 ms, total: 5.76 s
Wall time: 5.8 s


In [5]:
%%time
# Or using the built-in iterator
def print_thread(thread: pallas.Thread):
    for (token, index) in thread:
        print_pallas_object(token, index)

print_thread(trace.archives[0].threads[0])
print("Done ! It's definitely faster.")

Done ! It's definitely faster.
CPU times: user 1.13 s, sys: 9 μs, total: 1.13 s
Wall time: 1.13 s
